<p>23 de febrero de 2024</p>
<p>Universidad Autónoma de Aguascalientes</p>
<p>Departamento: Ciencias de la computación</p>
<p>Carrera: Ingeniería en Computación Inteligente</p>
<p>Maestro: Dr. Francisco Javier Luna Rosas</p>
<p>Curso: Machine learning y deep learning</p>
<p>Alumno: Antonio Muñoz Barrientos</p>
<p>Semestre Enero - junio 2024</p>

# Librerías

In [9]:
import cv2

### Leer la imagen y pasarla a matriz de valores

In [3]:

imagen_cheque = cv2.imread('Firmas/Firma1.jpg',0)
imagen_cheque

array([[240, 240, 242, ..., 254, 254, 255],
       [238, 238, 240, ..., 255, 255, 255],
       [243, 243, 244, ..., 254, 253, 253],
       ...,
       [239, 240, 248, ..., 254, 252, 248],
       [241, 242, 246, ..., 251, 249, 246],
       [252, 254, 255, ..., 255, 255, 255]], dtype=uint8)

### Recortamos la imagen en 4 partes y obtenemos solo la parte de la firma

In [4]:
alto, largo = imagen_cheque.shape

print("ancho {}, largo {}.".format(alto,largo))

# Dividir en cuatro cuadrantes
mitad_alto = int(alto / 2)
mitad_largo = int(largo / 2)

imagen_cheque = imagen_cheque[mitad_alto-10:alto, mitad_largo-10:largo]

ancho 261, largo 622.


### Obtenemos la imagen binaria (Blanco y negro)

In [5]:
_, umbral = cv2.threshold(imagen_cheque, 127, 255, cv2.THRESH_BINARY)
cv2.imwrite('Imagen binarizada.jpg',umbral)


True

### Buscamos TODOS los contornos

In [6]:
contornos, _ = cv2.findContours(umbral, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contornos

(array([[[  0,   0]],
 
        [[  0, 140]],
 
        [[320, 140]],
 
        [[320,   0]]], dtype=int32),
 array([[[  5, 118]],
 
        [[  6, 117]],
 
        [[  7, 118]],
 
        [[  6, 119]]], dtype=int32),
 array([[[  2, 118]],
 
        [[  3, 117]],
 
        [[  4, 118]],
 
        [[  3, 119]]], dtype=int32),
 array([[[  6, 117]],
 
        [[  7, 116]],
 
        [[  8, 117]],
 
        [[  7, 118]]], dtype=int32),
 array([[[  1, 117]],
 
        [[  2, 116]],
 
        [[  3, 117]],
 
        [[  2, 118]]], dtype=int32),
 array([[[  6, 113]],
 
        [[  7, 112]],
 
        [[  8, 113]],
 
        [[  8, 115]],
 
        [[  7, 116]],
 
        [[  6, 115]]], dtype=int32),
 array([[[  5, 112]],
 
        [[  6, 111]],
 
        [[  7, 112]],
 
        [[  6, 113]]], dtype=int32),
 array([[[  2, 112]],
 
        [[  3, 111]],
 
        [[  4, 111]],
 
        [[  5, 112]],
 
        [[  4, 113]],
 
        [[  3, 113]]], dtype=int32),
 array([[[292,  99]],
 
        

<p>Ordenamos los contornos, entre más area del contorno mayor contenido tiene, por lo tanto, se espera que la firma sea el contorno mas grande por detras del contorno general.</p>

In [7]:
contornos = sorted(contornos, key=cv2.contourArea, reverse=True)
len(contornos)

89

### Checar todos los contornos y verificar que sea la firma

<p>Vamos a recorrer todos los contornos obtenidos y checar si son parte de la firma. Como ordenamos anteriormente todos los contornos, esperamos que los primeros contornos, que son los más grandes, sean parte de la firma. Es posible que algunas firmas tengan elementos separados, entonces debemos utilizar alguna forma para detectar si son parte de la firma o es otro elemento del cheque.</p>
<p>Para poder detectar si es parte de la firma, tomamos en cuenta las coordenadas (x,y) y las posiciones de su perímetro (h,w). Designamos un rango de error para determinar que tanto podemos dejar variar estos valores para determinar si son parte de la firma o no (defualt 7). En cuanto algun contorno no cumpla esta condición se determina que la firma ya está terminada. </p>

In [8]:
media_anterior = False
median_index_error = 7
imagen_contornos = imagen_cheque.copy()
for contorno in contornos:
    x, y, w, h = cv2.boundingRect(contorno)
    #Checar que no sea borde de imagen
    if(x <= 3 or y <= 3):
        continue
    #Checar que sea primera parte de firma
    if(not media_anterior):
        media_anterior = int((y+w+h)/3)
        x_anterior = x
        y_anterior = y
        xmax = x
        ymax = y
        xmin = x_anterior
        ymin = y_anterior
        h_actual = h
        w_actual = w
        cv2.drawContours(imagen_contornos, [contorno], -1, (157, 255, 36), 3)
    #Checar si es siquiente parte de la firma
    else:
        media = int((y+w+h)/3)
        if(abs(media-media_anterior) <= median_index_error):
            xmin = x_anterior
            ymin = y_anterior
            xmax = x
            ymax = y
            h_actual = h
            w_actual = w
            cv2.drawContours(imagen_contornos, [contorno], -1, (157, 255, 36), 3)
            media_anterior = media
        else:
            #Obtener la firma
            firma = umbral[ymin:ymax+h_actual, xmin:xmax+w_actual]
            break

cv2.imwrite('Imagen con Contorno de Firma.jpg', imagen_contornos)
cv2.imwrite('Firma Extraída.jpg', firma)

True

# Conclusiones

Gracias a técnicas de procesamiento de imágenes como la binarización y la búsqueda de contornos podemos extraer de cualquier documento una firma. Es importante que debemos tratar de obtener todo el contenido de la firma incluso si esta es de varias partes, gracias a la función construida podemos obtener varias partes de las firmas. 

## Referencias

<p>CV2: Guía maestra OpenCV para desarrolladores de Python: <a href="https://konfuzio.com/es/cv2/"> https://konfuzio.com/es/cv2/</a></p>
<p>OpenCV-Python Tutorials: <a href="https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html">https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html</a></p>